In [ ]:
#LDA & genre1 KNN -> SBERT-FAISS

#기본 세팅

In [ ]:
pip install xlrd

In [ ]:
pip install sentence-transformers faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 11.7 MB/s eta 0:00:00


In [ ]:
pip install konlpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 72.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 488.6/488.6 kB 27.4 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import requests
import re
import konlpy
from konlpy.tag import Okt
from gensim import corpora
from gensim.corpora import Dictionary
from gensim.models import LdaModel

#데이터 불러오기

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


원본데이터

In [ ]:
df_original = pd.read_csv("/content/drive/MyDrive/알라딘 베스트셀러 Top 1000 1999~2024/가공 데이터/추천서비스_최종데이터(10.26).csv",
            encoding = 'utf-8-sig',
            dtype = {'ISBN' : "str"})
df = df_original.copy()

#CID group 하기

In [ ]:
cid_data = pd.read_excel("/content/drive/MyDrive/알라딘 베스트셀러 Top 1000 1999~2024/가공 데이터/aladin_Category_CID_20210927.xls")

#열 재정의, 필요없는 행 삭제
cid_data.columns = cid_data.iloc[1]
cid_data = cid_data.drop(index = [0,1])
cid_data.head(3)

1,CID,카테고리명,몰,1Depth,2Depth,3Depth,4Depth,5Depth
2,1230,가정/요리/뷰티,국내도서,가정/요리/뷰티,NaN,NaN,NaN,NaN
3,90452,가계부,국내도서,가정/요리/뷰티,가계부,NaN,NaN,NaN
4,53471,건강요리,국내도서,가정/요리/뷰티,건강요리,NaN,NaN,NaN


In [ ]:
grouped_cid = cid_data.groupby(['1Depth']).agg({'CID': lambda x: list(x)}).reset_index()

print("[1Depth] groupby 전체 length : ", len(grouped_cid))
print("CID가 하나만 속하는 데이터 :", len(grouped_cid[grouped_cid['CID'].apply(len) == 1]))
grouped_cid[50:60]

[1Depth] groupby 전체 length :  111
CID가 하나만 속하는 데이터 : 11


1,1Depth,CID
50,사전/기타,"[38419, 39068, 41818, 41819, 39200, 42527, 425..."
51,사회과학,"[798, 51052, 50999, 51121, 51117, 51111, 69510..."
52,상품권,[28640]
53,성인,[6548]
54,소설/시/희곡,"[1, 50930, 89481, 89482, 50922, 50927, 51116, ..."
55,수험서,"[90843, 97459, 97460, 97461, 97462, 97463, 974..."
56,수험서/자격증,"[1383, 34582, 34612, 34613, 34625, 34626, 3462..."
57,스페인 도서,"[28254, 159297, 75161, 75167, 75174, 75172, 75..."
58,알라딘오디오북,"[158585, 167553, 175974, 167547, 176194, 16754..."
59,애니메이션,"[6550, 69740, 55652, 56376, 89266, 66606, 6973..."


In [ ]:
#빠른 매핑을 위한 딕셔너리 만들기
depth2_dict = {}
for i, items in enumerate(grouped_cid['CID']):
  for id in items:
    depth2_dict[id] = grouped_cid['1Depth'].index[i]

#적용
df['genre_code'] = df['genre_id'].map(depth2_dict).astype('Int32')
df['genre_code'] = df['genre_code'].fillna(0).astype('Int32')
df.head(1)

In [ ]:
#test : 분류에 쓰일 데이터들
#new : 새로운 사용자 기록

df_test = df[['ISBN','genre_code', 'lda_result', '요약']]

#도서 검색
new_data에 [group 된 CID, lda 결과, 요약된 책 소개]

In [ ]:
#검색어
query = "조셉 머피"

In [ ]:
TTBKey = "ttbleejunho09091938001"

url = f"http://www.aladin.co.kr/ttb/api/ItemSearch.aspx?ttbkey={TTBKey}&Query={query}&QueryType=Title&MaxResults=10&start=1&SearchTarget=Book&output=JS&Version=20131101"
res = requests.get(url)
res.raise_for_status()

original_cid = res.json()['item'][0]['categoryId']
original_description = res.json()['item'][0]['description']

In [ ]:
#1. CID
grouped_cid = depth2_dict[original_cid]

In [ ]:
#2. LDA
#new_description 전처리
new_description = re.sub(r'[^\w\s가-힣]', ' ', original_description)  # 특수문자 제거
new_description = re.sub(r'[\n\d]', ' ', new_description)  # 줄바꿈, 숫자 제거

#Okt, 불용어사전 준비
Okt = konlpy.tag.Okt()
stopword_data_path = '/content/drive/MyDrive/알라딘 베스트셀러 Top 1000 1999~2024/가공 데이터/stopwords-ko.txt'
with open(stopword_data_path, 'r', encoding='utf-8') as f:
  stopwords = [line.strip() for line in f.readlines()]

#Okt, 불용어제거 실행
Okt_morphs = Okt.pos(new_description)
new_description = ' '.join([word for word, pos in Okt_morphs if pos == 'Noun' and word not in stopwords and len(word)>1])
new_description

'잠재의식 머피 모든 지식 통찰 망라 머피 잠재의식 고전 시리즈 대미 장식 마지막 머피 잠재의식 고전 시리즈 머피 트러스트 공식 저서 펭귄 랜덤하우스 주제 별로 편집'

In [ ]:
# 저장된 LDA 모델을 불러오기
lda_model = LdaModel.load('/content/drive/MyDrive/알라딘 베스트셀러 Top 1000 1999~2024/가공 데이터/lda model/lda_model.model')

# 저장된 dictionary 파일을 불러오기
dictionary = Dictionary.load('/content/drive/MyDrive/알라딘 베스트셀러 Top 1000 1999~2024/가공 데이터/lda model/dictionary.dict')

# 저장된 corpus 파일을 불러오기
corpus = corpora.MmCorpus('/content/drive/MyDrive/알라딘 베스트셀러 Top 1000 1999~2024/가공 데이터/lda model/corpus.mm')

In [ ]:
new_doc = new_description.split() #토큰화
new_doc2bow = dictionary.doc2bow(new_doc) #doc2bow
new_doc_lda = lda_model.get_document_topics(new_doc2bow) #lda 분석

for topic_id, prob in new_doc_lda:
    print(f"토픽 {topic_id}: 확률 {prob}")
#최대 확률 토픽
max_topic_id = max(new_doc_lda, key=lambda x: x[1])[0]

lda_topic = max_topic_id #저장
print(lda_topic)

토픽 1: 확률 0.16310614347457886
토픽 7: 확률 0.05599827319383621
토픽 21: 확률 0.2057473063468933
토픽 25: 확률 0.0539204478263855
토픽 54: 확률 0.05765799432992935
토픽 67: 확률 0.30314937233924866
토픽 83: 확률 0.03229124844074249
토픽 88: 확률 0.07557599991559982
67


In [ ]:
new_data = [grouped_cid, lda_topic, original_description]
print(new_data)

[81, 67, '잠재의식에 관한 조셉 머피의 모든 지식과 통찰이 총망라된 ‘조셉 머피 잠재의식의 고전’ 시리즈의 대미를 장식한 마지막 책이다. ‘조셉 머피 잠재의식의 고전’ 시리즈는 머피 트러스트에서 인정받은 유일한 공식 저서로, 펭귄랜덤하우스에서 출간한 10권을 각각 주제별로 묶어 5권으로 재편집한 것이다.']


#KNN - genre_code, lda_result
genre_code, lda_result 이용한 knn 유사도로 비슷한거 나열

In [ ]:
from sklearn.neighbors import NearestNeighbors

classifier = NearestNeighbors(n_neighbors = 100, metric='euclidean').fit(df_test[['lda_result','genre_code']])
distances, indices = classifier.kneighbors([new_data[:2]])

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(


In [ ]:
print("유사한 항목 (가까운 순서대로):")
df_test2 = pd.DataFrame(columns=df_test.columns)

for idx, distance in zip(indices[0], distances[0]):
  #print(f"인덱스: {idx}, 거리: {distance}\n데이터: {df_test.iloc[idx]}\n\n")

  a = df_test.iloc[idx]

  df_test2.loc[len(df_test2)] = [a['ISBN'], a['genre_code'], a['lda_result'], a['요약']]



유사한 항목 (가까운 순서대로):


#책 소개 SBERT - FAISS 분석
책 소개 데이터 Sentence BERT로 임베딩 후 FAISS분석

S-BERT로 임베딩

In [ ]:
import tensorflow as tf
from sentence_transformers import SentenceTransformer
import numpy as np
import faiss

# 2. Sentence-BERT 모델 로드
model = SentenceTransformer('all-MiniLM-L6-v2')  # 임베딩용 모델 선택

# 3. 임베딩 생성 (정수 결합 + 문장 임베딩)
def create_embedding(data):
    sentence_embedding = model.encode(data)  # 문장 임베딩

    return sentence_embedding

# 전체 데이터 임베딩 생성
embeddings = create_embedding(df_test2['요약'])

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


FAISS 사용

In [ ]:
# 4. FAISS 인덱스 생성
d = embeddings.shape[1]  # 벡터 차원
index = faiss.IndexFlatL2(d)  # L2 거리 기반 인덱스 생성
index.add(embeddings)  # 임베딩 데이터 추가

# 5. 새로운 데이터로 유사한 데 이터 검색
def find_similar_data(sentence, top_k=1):
  query_embedding = model.encode(sentence).reshape(1,-1) #-1은 자동으로 계산하라는 뜻.

  _, similar_indices = index.search(query_embedding, top_k)  # 유사도 검색
  return similar_indices

# 테스트
similar_data = find_similar_data(new_data[2], top_k=3)

for i in similar_data[0]:
  print(df_test2.iloc[i])

ISBN                                             9788983944627 
genre_code                                                   64
lda_result                                                   84
요약              소심하고 겁 많고 까탈스러운 여자 혼자 떠나는 걷기 여행 의 저자 도보여행가 김...
Name: 82, dtype: object
ISBN                                             9791192444352 
genre_code                                                   65
lda_result                                                   83
요약            이집트에 관심이 있는 사람들을 위한 입문서가 출간되었습니다. 이 책은 한 책으로 모...
Name: 19, dtype: object
ISBN                                             9788932474915 
genre_code                                                   66
lda_result                                                   84
요약            윤광준의 생활명품 시리즈의 궁극 에디션인 ‘윤광준의 생활명품 101’은 유효한 도구...
Name: 20, dtype: object


#결과 확인용

In [ ]:
print("1. 기록한 도서")
print("title : ", res.json()['item'][0]['title'])
print("description : ", res.json()['item'][0]['description'])

print()
print()

print("2. 결과")
for i in similar_data:
  a = df_test2.iloc[i]['ISBN'].values

for ISBN in a:
  new_ISBN = ISBN
  url = f"http://www.aladin.co.kr/ttb/api/ItemLookUp.aspx?ttbkey={TTBKey}&itemIdType=ISBN13&ItemId={new_ISBN}&output=JS&Version=20131101"
  res = requests.get(url)
  res.raise_for_status()
  print(res.json()['item'][0]['title'])
  print(res.json()['item'][0]['description'])
  print()



1. 기록한 도서
title :  조셉 머피 잠재의식의 힘 - 삶의 기적을 일으키는 내면의 보물창고
description :  잠재의식에 관한 조셉 머피의 모든 지식과 통찰이 총망라된 ‘조셉 머피 잠재의식의 고전’ 시리즈의 대미를 장식한 마지막 책이다. ‘조셉 머피 잠재의식의 고전’ 시리즈는 머피 트러스트에서 인정받은 유일한 공식 저서로, 펭귄랜덤하우스에서 출간한 10권을 각각 주제별로 묶어 5권으로 재편집한 것이다.


2. 결과
유럽의 걷고 싶은 길 - 도보여행가 김남희가 반한
&lt;소심하고 겁 많고 까탈스러운 여자 혼자 떠나는 걷기 여행&gt의 저자 도보여행가 김남희의 다섯 번째 여행기. 저자가 2007년 스페인에 머무는 동안 이탈리아, 스페인, 프랑스, 영국, 잉글랜드의 '걷기 좋은 길'을 틈틈이 여행하며 쓴 에세이다. 김남희는 도보여행자들의 천국인 유럽에서도 가장 아름답고 호젓하기로 유명한 '길'을 걸었다.

고대 이집트 해부도감
국내 최대 규모의 ‘이집트 미라전’이 주목받으며 준비되고 있고, 이집트 일주가 여행객들의 인기 여행상품으로 자리 잡는 등 북아프리카 이집트에 대한 대중적 관심이 점차 높아지고 있다. 이런 가운데 4대 문명의 정점인 고대 이집트에 관한 모든 것을 쉽고 친절하게 한 권으로 정리 해설한 입문서가 국내에서 출간되었다.

윤광준의 생활명품 101
윤광준 작가가 엄선한 일상에서 유의미하고 아름다운 물건 101가지를 담은 『윤광준의 생활명품 101』이 을유문화사에서 출간되었다. 화분부터 뵈젠도르퍼 임페리얼 피아노까지 몸과 생활공간을 풍요롭고 디테일하게 가꿔 주는 도구 101가지가 들어 있다.

